## Market Espresso

In [2]:
# Setting up notebook
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px, plotly.io as pio
import matplotlib.pyplot as plt
import datetime as dt
import smtplib
import os
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email import encoders

import sys
set_wd = '/Users/Anthony/Desktop/VSFolder'
sys.path.append(set_wd)

from pws import my_email as email, my_email_pw as email_pw


In [3]:
# S&P 500 ticker symbol
# sp500_symbol = "^GSPC"
# dow_symbol = "^DJI"
# nasdaq_symbol = "^IXIC"

# Dictionary of ticker symbols and stock/index name
ticker_dict = {
    '^GSPC':{'Name':'S&P 500'}, 
    '^DJI':{'Name':'Dow'}, 
    '^IXIC':{'Name':'Nasdaq'},
    'VNQ':{'Name':'Vanguard REI ETF'},
    'FXAIX':{'Name':'Fidelity 500 Index Fund'},
    'TSLA':{'Name':'Tesla'}
    }

# Getting lists of ticker symbols and names for future use
ticker_symbols = list(ticker_dict.keys())
ticker_names = [ticker_dict[s]['Name'] for s in ticker_symbols]

# Setting time period
period = '3mo'
rolling_window = 5

# Get latest S&P 500 index data from Yahoo Finance
# # This creates three Ticker objects
# sp500_ticker = yf.Ticker(sp500_symbol)
# dow_ticker = yf.Ticker(dow_symbol)
# nasdaq_ticker = yf.Ticker(sp500_symbol)


In [176]:
# Investigating the ticker object
# print(type(sp500_ticker))
# dir(sp500_ticker) -- gives the available methods in ticker object

# ticker_names



In [4]:
# Function to prep ticker data

def ticker_df(symbol, period = period):

    df = yf.Ticker(symbol).history(period = period).reset_index()

    # Formats dates and gets name of index/stock
    df["Name"] = ticker_dict[symbol]['Name']
    df["Ticker Symbol"] = symbol
    df["Date"] = df["Date"].dt.date

    # Formats values
    df["Open"] = df["Open"].round(0)
    df["High"] = df["High"].round(0)
    df["Low"] = df["Low"].round(0)
    df["Close"] = df["Close"].round(0)

    # Computing additional metrics
    df["Daily Range"] = df["High"] - df["Low"]
    df["Daily Change"] = df["Close"] - df["Open"]
    df["Daily Change (%)"] = ((df["Close"] - df["Open"])/df["Open"] * 100).round(1)
    df["Prior 1D Close"] = df["Close"].shift(1)
    df["DoD Close Delta"] = df["Close"] - df["Prior 1D Close"]
    df["DoD Pct Change"] = ((df["Close"]/df["Prior 1D Close"] - 1) * 100).round(2)
    df["DoD Large Change Flag"] = df["DoD Pct Change"].apply(lambda p: 1 if abs(p) >= 0.01 else 0)
    df["DoD Pct Change"] = df["DoD Pct Change"].apply(lambda p: f'{p:.1f}%')
    df["Close Lower 1D Lag"] = df["DoD Close Delta"].apply(lambda x: 1 if x < 0 else 0)
    df["Close Lower 2D Lag"] = df["Close Lower 1D Lag"].shift(1)
    df["Close Lower 3D Lag"] = df["Close Lower 1D Lag"].shift(2)
    df["3D Decr Flag"] = df.apply(lambda row: 0 if row["Close Lower 1D Lag"] + row["Close Lower 2D Lag"] + row["Close Lower 3D Lag"] < 3 else 1, axis = 1)
    df[f"{rolling_window}D Close Rolling Avg"] = df["Close"].rolling(rolling_window).mean().round(0)

    # Formatting volume
    df["Volume (Millions)"] = df["Volume"].apply(lambda v: v/1000000).round(1)
    df.drop(["Volume"], axis = 1, inplace = True)

    # Storing df in dictionary for each symbol
    ticker_dict[symbol]["Dataframe"] = df

    return df

ticker_df_all = pd.concat([ticker_df(t) for t in ticker_symbols], ignore_index = True)



In [6]:
# Setting working directory to where we want to store the images produced
set_dir = '/Users/Anthony/Desktop/VSFolder/Financial Analytics/Market-Analytics/Chart Images/'
os.chdir(set_dir)

In [7]:
# Creating an interactive line chart with plotly

def tracker(smbl, y_vars = ['Low','Close','High'], hover_list = ['Date']):
    name = ticker_dict[smbl]['Name']
    x_axis = 'Date'
    y_axis = y_vars
    # y_axis = ["5D Close Rolling Avg"]

    x_label = 'Date'
    y_label = f'{name}'

    fig = px.line(ticker_df_all[ticker_df_all["Ticker Symbol"] == smbl], x = x_axis, y = y_axis, title = f'{name} - H/L/Close', hover_data = hover_list, line_shape = 'spline')
    fig.update_xaxes(title_text = x_label)
    fig.update_yaxes(title_text = y_label)

    pio.write_image(fig,f'{name}_tracker.png',format = 'png')
    chart_file_name = f'{name}_tracker.png'
    # fig.show()

    return fig, chart_file_name

In [8]:
ticker_df_all[['Name','Low', 'High', 'Open', 'Close', 'Daily Range','Daily Change', 'Daily Change (%)']]

,Name,Low,High,Open,Close,Daily Range,Daily Change,Daily Change (%)
0,S&P 500,5281.0,5316.0,5316.0,5306.0,35.0,-10.0,-0.2
1,S&P 500,5263.0,5282.0,5279.0,5267.0,19.0,-12.0,-0.2
2,S&P 500,5222.0,5260.0,5260.0,5235.0,38.0,-25.0,-0.5
3,S&P 500,5192.0,5280.0,5243.0,5278.0,88.0,35.0,0.7
4,S&P 500,5234.0,5302.0,5297.0,5283.0,68.0,-14.0,-0.3
...,...,...,...,...,...,...,...,...
372,Tesla,220.0,228.0,225.0,221.0,8.0,-4.0,-1.8
373,Tesla,219.0,225.0,223.0,223.0,6.0,0.0,0.0
374,Tesla,210.0,225.0,224.0,211.0,15.0,-13.0,-5.8
375,Tesla,214.0,221.0,214.0,220.0,7.0,6.0,2.8


In [9]:
# Getting minima and maximua over the time period
ticker_summary = ticker_df_all.groupby(['Ticker Symbol'])[['Open','Daily Range','Daily Change (%)','Close']].agg(['min','max','median','mean'])

# minsCL = ticker_df_all.groupby(['Ticker Symbol']).min()['Close']
# maxsCL = ticker_df_all.groupby(['Ticker Symbol']).max()['Close']
# avgCL = ticker_df_all.groupby(['Ticker Symbol']).mean()['Close']

# Note: We always take the latest date at which the security took on its max/min values
for s in ticker_symbols:

    # Creating summary entry in dictionary for recall
    ticker_dict[s]["Summary"] = ticker_summary.loc[s]

    # Summarizing opening data
    ticker_dict[s]["Min Open Value"] = ticker_summary.loc[s,'Open']['min']
    ticker_dict[s]["Min Open Date"] = ticker_df_all[(ticker_df_all['Ticker Symbol'] == s) & (ticker_df_all['Open'] == ticker_dict[s]["Min Open Value"])]['Date'].max()
    ticker_dict[s]["Max Open Value"] = ticker_summary.loc[s,'Open']['max']
    ticker_dict[s]["Max Open Date"] = ticker_df_all[(ticker_df_all['Ticker Symbol'] == s) & (ticker_df_all['Open'] == ticker_dict[s]["Max Open Value"])]['Date'].max()
    ticker_dict[s]["Avg Open Value"] = ticker_summary.loc[s,'Open']['mean'].round(1)
    ticker_dict[s]["Med Open Value"] = ticker_summary.loc[s,'Open']['median']

    # Summarizing close data
    ticker_dict[s]["Min Close Value"] = ticker_summary.loc[s,'Close']['min']
    ticker_dict[s]["Min Close Date"] = ticker_df_all[(ticker_df_all['Ticker Symbol'] == s) & (ticker_df_all['Close'] == ticker_dict[s]["Min Close Value"])]['Date'].max()
    ticker_dict[s]["Max Close Value"] = ticker_summary.loc[s,'Close']['max']
    ticker_dict[s]["Max Close Date"] = ticker_df_all[(ticker_df_all['Ticker Symbol'] == s) & (ticker_df_all['Close'] == ticker_dict[s]["Max Close Value"])]['Date'].max()
    ticker_dict[s]["Avg Close Value"] = ticker_summary.loc[s,'Close']['mean'].round(1)
    ticker_dict[s]["Med Close Value"] = ticker_summary.loc[s,'Close']['median']

    # Summarizing daily range and pct change data
    ticker_dict[s]["Avg Pct Change"] = ticker_summary.loc[s,'Daily Change (%)']['mean'].round(1)
    ticker_dict[s]["Med Pct Change"] = ticker_summary.loc[s,'Daily Change (%)']['median']
    ticker_dict[s]["Avg Daily Range"] = ticker_summary.loc[s,'Daily Range']['mean'].round(1)
    ticker_dict[s]["Med Daily Range"] = ticker_summary.loc[s,'Daily Range']['median']

    # Adding the graph over the time period, saving the chart as an image to show in email
    ticker_dict[s]["Graph"], ticker_dict[s]["Chart"] = tracker(s, y_vars = ['Low','Close','High'], hover_list = ['Date'])


# Creating a list of chart image file names to attach to email
ticker_charts = [ticker_dict[s]["Chart"] for s in ticker_symbols]
    

In [202]:
# # creating line chart to display close over time
# plt.figure(figsize=(10, 6))
# plt.plot(ticker_df_all["Date"], ticker_df_all["Close"], marker='o', linestyle='-', color='b')
# plt.xlabel('Date')
# plt.ylabel('S&P 500 Close')
# plt.title('S&P 500 Close - Last 3 Months')
# plt.grid(True)
# plt.xticks(rotation=45)  # Rotate dates for better readability
# plt.tight_layout()


# # Show plot
# plt.show()

In [209]:
# Sending emails with attachment

# Email and SMTP configuration
sender_email = email
receiver_emails = [email]
password = email_pw
smtp_server = 'smtp.gmail.com'
smtp_port = 587

# Create a multipart message
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = ''
msg['Bcc'] = ';'.join(receiver_emails)
msg['Subject'] = f"Market Espresso Shot - {dt.datetime.now().strftime('%B %d, %Y')}"

# Add HTML body to the email
email_body = 'Testing this out'
# email_body = ''
msg.attach(MIMEText(email_body, 'html'))

# Attaching charts/media to emails
for file in ticker_charts:
    with open(file, 'rb') as image:
        part = MIMEBase('application','octet-stream')
        part.set_payload(image.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename = {os.path.basename(file)}')
        msg.attach(part)

# Connect to the SMTP server and send the email
try:
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_emails, msg.as_string())
    print("Email sent successfully!")
except Exception as e:
    print(f"Failed to send email. Error: {str(e)}")
finally:
    server.quit()

Email sent successfully!
